# Bisecting - halvering i Julia



In [13]:
#total.db har en tabell "hyphendata" indeksert, og en uindeksert "words"

db_file = "/mnt/disk1/Github/total.db"

"/mnt/disk1/Github/total.db"

### Importer moduler fra Julia

In [56]:
using SQLite;
using DataFrames;
using PyFormattedStrings;

### Åpne en kobling

In [16]:
db = SQLite.DB(db_file)

SQLite.DB("/mnt/disk1/Github/total.db")

In [200]:
SQLite.tables(db) |> DataFrame

Row,name,schema
,String,Schema
1,hyph,"Tables.Schema:\n :first Union{Missing, String}\n :second Union{Missing, String}"
2,hyphendata,"Tables.Schema:\n :first Union{Missing, String}\n :second Union{Missing, String}\n :word Union{Missing, String}\n :freq Union{Missing, Int64}"
3,words,"Tables.Schema:\n :word Union{Missing, String}\n :freq Union{Missing, Int64}"


### Wrapper 

Ikke noe stort poeng i wrapperen, men den stammer fra Python-koden

In [37]:
function query(db, sql, params=())
        data = SQLite.DBInterface.execute(db, sql, params) |> DataFrame
    return data
end

query (generic function with 2 methods)

## Halveringsalgoritme med rowid

Definerer to funksjoner, en toppnivå, og en som gjør det rekursive (iterative) arbeidet

I praksis må toppnivået justere radstart og radslutt

In [206]:
function bisectiterate(db, word)
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[!,"rowid"][1]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[!, "rowid"][1]
    
    return bisect(db, word, minrow, maxrow)
end

bisectiterate (generic function with 1 method)

## bisect

Funksjonen `bisect` går gjennom en sortert tabell uten indeks. Søket starter ved å sjekke midtpunktet på tabellen, hvis det ikke er treff går den til neste halvdel, avhengig av om ordet som er funnet er større eller mindre enn det vi søker etter. Ved treff, scannes tabellen for tilsvarende ord før og etter.

In [208]:
function bisect(db, word, minrow, maxrow)

    found = false
    from_row = minrow
    to_row = maxrow
    res = DataFrame()
    i = 0
    delta = 0
    limit = log2(maxrow + 1)
    
    while !found && (from_row <= to_row) && i < limit
        
        i += 1
        
        mid_row = from_row + div(to_row - from_row, 2)
        
        targeted = query(db, f"select word, freq, rowid from words where rowid={mid_row}")
        
        if cmp(word,targeted[!,"word"][1]) == 0
            #println(targeted[!,"word"][1])
            append!(res, targeted)
            
            found = true

            left_candidate = mid_row - 1
            l_candidate = query(db, f"select word, freq, rowid from words where rowid={left_candidate}")
            
            while cmp(l_candidate[!,"word"][1], word) == 0 && left_candidate >= from_row
                append!(res, l_candidate)
                left_candidate -= 1
                l_candidate = query(db, f"select word, freq, rowid from words where rowid={left_candidate}")
            end
            
            right_candidate = mid_row + 1
            r_candidate = query(db, f"select word, freq, rowid from words where rowid={right_candidate}")
            
            while cmp(r_candidate[!,"word"][1], word) == 0 && right_candidate <= to_row
                append!(res, r_candidate)
                right_candidate += 1
                r_candidate = query(db, f"select word, freq, rowid from words where rowid={right_candidate}")
            end
            
        elseif cmp(word, targeted[!,"word"][1]) == 1
            to_row = mid_row
        else
            from_row = mid_row 
        end
        
    end
    return res
end

bisect (generic function with 2 methods)

# Timing

Såvidt jeg kan se klarer Julia å henge med på SQLite. 10 ganger raskere enn Python. Men for full tablescan er hastigheten den samme (ca. 12 sek) så det er når det går fort at Julia skinner. 

## Bisect

In [211]:
@time begin
    bisectiterate(db, "vårrengjøring")
end

  0.001429 seconds (3.39 k allocations: 171.688 KiB)


Row,word,freq,rowid
,String,Int64,Int64
1,vårrengjøring,331,11309440
2,vårrengjøring,644,11309439
3,vårrengjøring,298,11309441
4,vårrengjøring,60,11309442
5,vårrengjøring,25,11309443


## Full table-scan

In [205]:
@time begin
    query(db, "select word,freq from words where word = 'Nasjonalbiblioteket' ")
end

 12.777626 seconds (168 allocations: 8.297 KiB)


Row,word,freq
,String,Int64
1,Nasjonalbiblioteket,2276
2,Nasjonalbiblioteket,1816
3,Nasjonalbiblioteket,1372
4,Nasjonalbiblioteket,1357
5,Nasjonalbiblioteket,1301
6,Nasjonalbiblioteket,1108
7,Nasjonalbiblioteket,419
8,Nasjonalbiblioteket,398
9,Nasjonalbiblioteket,39


## Indekserte data

Her er tiden det tar å hente ut de indekserte dataene, som er ca 10 gangen igjen, tilsvarende som for Python, men Python er 10 ganger tregere. Forholdet mellom Pythons bisect og indeksert er røfflig den samme

In [203]:
@time begin
    query(db, "select word, freq from hyphendata where word = 'Nasjonalbiblioteket'")
end

  0.000246 seconds (168 allocations: 8.297 KiB)


Row,word,freq
,String,Int64
1,Nasjonalbiblioteket,2
2,Nasjonalbiblioteket,12
3,Nasjonalbiblioteket,14
4,Nasjonalbiblioteket,39
5,Nasjonalbiblioteket,398
6,Nasjonalbiblioteket,419
7,Nasjonalbiblioteket,1108
8,Nasjonalbiblioteket,1301
9,Nasjonalbiblioteket,1357
